In [1]:
import random
import numpy as np
import pandas as pd
from copy import copy, deepcopy
from scipy.optimize import minimize
import itertools

import plotly
import plotly.graph_objs as go

import plotly.express as px
from plotly.subplots import make_subplots

from pathlib import Path
import json
import os

import sys
import time

In [2]:
# Возвращает функцию гаусса с заданными параметрами
def get_gaus_f(x,y,d):
    return lambda t: y*np.exp(-(t-x)**2/2/d**2)



# Сортирует информацию о гауссах в решении в возрастающем порядке
# На вход - матрица решений [[x1,A1,disp1], ...]
# На выход - отсортированная матрица по x [[xn,An,dispn], ...]
def Sort_gaus(v_sol):
    m_sol = np.array(v_sol).reshape(-1,3)
    m_sol = np.array(sorted(m_sol, key = lambda gaus: gaus[0]))
    v_sol_sorted = m_sol.reshape(-1).tolist()
    return v_sol_sorted


# Возвращает спектр по параметрам гауссовых кривых
def EvalSol(sol, l_wavelength = np.linspace(0 ,1 , 1024), add_noise_disp = 0, mult_noise_disp = 0, seed=None):
    X = l_wavelength
    spec = np.zeros(len(X))
    for n_gaus in range(0,len(sol), 3):
        f_gaus = get_gaus_f(sol[n_gaus], sol[n_gaus+1], sol[n_gaus+2])
        spec += np.array([f_gaus(t) for t in X])

    #Добавление шума
    random.seed(seed)
    add_spec_noise = np.random.normal(0, add_noise_disp, len(X))
    mult_spec_noise = np.random.normal(0, mult_noise_disp, len(X)) * spec
    spec = spec + add_spec_noise + mult_spec_noise

    return spec, X
    

# Рисует оба спектра и гауссианы
def DrawApprox(l_spec_1, l_spec_2, l_wavelength = np.linspace(0 ,1 , 1024), sol = np.array([])):
    fig = go.Figure()
    fig.update_layout(title_text="Spectrum",
                      title_font_size=20)
    
    K = l_wavelength
    fig.add_trace(go.Scatter(x=K,
                             y=l_spec_1,
                             name='1-st spec'))
    fig.add_trace(go.Scatter(x=K,
                             y=l_spec_2,
                             name='2-nd spec'))
    
    l_color = ['LimeGreen', 'MediumSlateBlue', 'Orange', 'DeepSkyBlue']
    i=0
    if len(sol) > 0:
        for n_gaus in range(0,len(sol), 3):
            f_gaus = get_gaus_f(sol[n_gaus], sol[n_gaus+1], sol[n_gaus+2])
            fig.add_vline(x=sol[n_gaus], line_width=2, line_color=l_color[i])
            fig.add_trace(go.Scatter(x=K,
                          y=[f_gaus(t) for t in K],
                          name=f'{n_gaus//3+1} gauss'))
            i+=1
    fig.show()
    return 


# Возвращает функцию Err(список-решение)
def GetFuncErr(spec, l_wavelength = np.linspace(0 ,1 , 1024)):
    F_err = lambda sol: np.sum((spec - EvalSol(sol)[0])**2)

    return F_err


def Grad_desc(f_Err, bounds, x0=None):
    if x0==None:
        x0=[]
        for a, b in bounds:
            x0.append(np.random.uniform(a,b))
        x0=np.array(x0)
        #print(f'x0: {x0}')

    res = minimize(f_Err, x0 = x0, bounds = bounds, method = 'SLSQP', options={'ftol':0})

    sol = res.x.tolist()
    f_err = f_Err(sol)

    return sol, f_err

In [3]:
disp_g = 0.1
ampl_gaus_noise = 0#0.2
add_noise_disp = 0.05

sol_1 = [0.3, 0.6, disp_g, 
         0.5, 0.8, disp_g,
         0.7, 0.7, disp_g,
         0.5, ampl_gaus_noise, 0.3]

points_spec_1, X = EvalSol(sol_1, add_noise_disp=add_noise_disp, seed=1)

F_err = GetFuncErr(points_spec_1, X)

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1],
          [0,1], [0,1], [0,1]]

sol_2, fin_err = Grad_desc(f_Err=F_err, bounds=bounds, x0=sol_1[:-3])

print(f'fin_err: {fin_err}\tsol: {sol_2}')

points_spec_2, X = EvalSol(sol_2)


DrawApprox(points_spec_1, points_spec_2, X, sol_2)
DrawApprox(points_spec_1, points_spec_2, X, sol_1)

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  return lambda t: y*np.exp(-(t-x)**2/2/d**2)
C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return lambda t: y*np.exp(-(t-x)**2/2/d**2)


fin_err: 2.6372719012045156	sol: [0.3079097009555208, 0.6339011011132115, 0.1023189960520349, 0.5031634517146546, 0.772845427116613, 0.09382598522708381, 0.6980032023887295, 0.7158640614952052, 0.09996966411296725]


# Эксперимент

In [17]:
def exper(init_sol, bounds, x0=None, 
          add_noise_disp = 0, mult_noise_disp = 0, seed=None):
    '''Решение задачи, созданной гауссианами в init_sol.
    '''
    init_spec, X = EvalSol(init_sol, 
                           l_wavelength=np.linspace(0,1,1024), 
                           add_noise_disp = add_noise_disp, 
                           mult_noise_disp = mult_noise_disp, 
                           seed=seed)
    F_err = GetFuncErr(init_spec, X)

    sol, fin_err = Grad_desc(f_Err=F_err, bounds=bounds, x0=x0)

    return sol, fin_err


def mult_exper(disp_g, ampl_gaus_noise, bounds, add_noise_disp = 0, mult_noise_disp = 0, seed=None, num_launches=1):
    '''Проводит поиск решения c заданными дисперсиями гауссовых кривых и уровнем гауссового шумом.
    С помощью нескольких запусков.
    '''
    init_sol = [0.3, 0.6, disp_g, 
                0.5, 0.8, disp_g,
                0.7, 0.7, disp_g,
                0.5, ampl_gaus_noise, 0.3]
    
    print(f'\n=== disp_g: {disp_g}\tampl_noise: {ampl_gaus_noise} ===')

    best_sol, best_fin_err = exper(init_sol, bounds, x0=init_sol[:-3], # 0-guess
                                   add_noise_disp = add_noise_disp, mult_noise_disp = mult_noise_disp, seed=seed)
    for i in range(num_launches):
        sol, fin_err = exper(init_sol, bounds,
                             add_noise_disp = add_noise_disp, mult_noise_disp = mult_noise_disp, seed=seed)
        #print(f'{i}.\terr: {fin_err}\tx0: {sol}')
        if fin_err<best_fin_err:
            best_sol, best_fin_err = sol, fin_err
    print(f'= best_fin_err: {best_fin_err} =')


    return best_sol, best_fin_err


def mult_exper_2fit(disp_g, ampl_gaus_noise, bounds, add_noise_disp = 0, mult_noise_disp = 0, seed=None, num_launches=1):
    '''Проводит поиск решения c заданными дисперсиями гауссовых кривых и аддитивным шумом.
    С помощью нескольких запусков.Возвращает невязку решения с исходным спектром с шумом и без шума.
    '''
    init_sol = [0.3, 0.6, disp_g, 
                0.5, 0.8, disp_g,
                0.7, 0.7, disp_g,
                0.5, ampl_gaus_noise, 0.3]
    
    print(f'\n=== disp_g: {disp_g}\tadd_noise: {add_noise_disp} ===')

    best_sol, best_fin_err = exper(init_sol, bounds, x0=init_sol[:-3], # 0-guess
                                   add_noise_disp = add_noise_disp, mult_noise_disp = mult_noise_disp, seed=seed)
    for i in range(num_launches):
        sol, fin_err = exper(init_sol, bounds,
                             add_noise_disp = add_noise_disp, mult_noise_disp = mult_noise_disp, seed=seed)
        #print(f'{i}.\terr: {fin_err}\tx0: {sol}')
        if fin_err<best_fin_err:
            best_sol, best_fin_err = sol, fin_err
    print(f'= best_fin_err: {best_fin_err} =')

    init_spec_no_noise, X = EvalSol(init_sol)
    F_err = GetFuncErr(init_spec_no_noise, X)
    best_fin_err_no_noise = F_err(best_sol)
    print(f'= best_fin_err_no_noise: {best_fin_err_no_noise} =')

    return best_sol, best_fin_err, best_fin_err_no_noise


def mult_exper_3(disp_g, ampl_gaus_noise, bounds, add_noise_disp = 0, mult_noise_disp = 0, seed=None, num_launches=1):
    '''Проводит поиск решения c заданными дисперсиями гауссовых кривых и аддитивным шумом.
    С помощью нескольких запусков. Возвращает невязку решения с исходным спектром с шумом и без шума, а также delta_x, delta_y, delta_s.
    '''
    init_sol = [0.3, 0.6, disp_g, 
                0.5, 0.8, disp_g,
                0.7, 0.7, disp_g,
                0.5, ampl_gaus_noise, 0.3]
    
    print(f'\n=== disp_g: {disp_g}\tampl_noise: {ampl_gaus_noise}\tadd_noise: {add_noise_disp} ===')

    best_sol, best_fin_err = exper(init_sol, bounds, x0=init_sol[:-3], # 0-guess
                                   add_noise_disp = add_noise_disp, mult_noise_disp = mult_noise_disp, seed=seed)
    best_sol, best_fin_err = None, np.inf
    for i in range(num_launches):
        sol, fin_err = exper(init_sol, bounds,
                             add_noise_disp = add_noise_disp, mult_noise_disp = mult_noise_disp, seed=seed)
        #print(f'{i}.\terr: {fin_err}\tx0: {sol}')
        if fin_err<best_fin_err:
            best_sol, best_fin_err = sol, fin_err
    print(f'= best_fin_err: {best_fin_err} =')

    init_spec_no_noise, X = EvalSol(init_sol[:-3])
    F_err = GetFuncErr(init_spec_no_noise, X)
    best_fin_err_no_noise = F_err(best_sol)
    print(f'= best_fin_err_no_noise: {best_fin_err_no_noise} =')

    best_sol_sorted = Sort_gaus(best_sol)
    #print(np.array(best_sol)-np.array(init_sol[:-3]))
    delta_x, delta_y, delta_s = (np.abs(np.array(best_sol_sorted)-np.array(init_sol[:-3]))).reshape([-1,3]).mean(axis=0)

    return best_sol_sorted, best_fin_err, best_fin_err_no_noise, delta_x, delta_y, delta_s

# 8-е приближение.
Вместо гаусового шума - аддитивный шум.

In [6]:
'''l_disp_g = [0.1, 0.2, 0.3]
l_ampl_noise = [0.05, 0.1, 0.15, 0.2]
'''

l_disp_g = np.linspace(0.075, 0.12, 16)
ampl_gaus_noise = 0
l_add_noise_disp = np.linspace(0, 0.05, 16)

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1],
          [0,1], [0,1], [0,1]]


l_errors = []
l_errors_no_noise = []
l_delta_x, l_delta_y, l_delta_s = [],[],[]

for disp_g, add_noise_disp in itertools.product(l_disp_g, l_add_noise_disp):
    best_sol, best_fin_err, best_fin_err_no_noise, delta_x, delta_y, delta_s = mult_exper_3(disp_g, ampl_gaus_noise, bounds = bounds, add_noise_disp = add_noise_disp, num_launches=15)
    l_errors.append(best_fin_err)
    l_errors_no_noise.append(best_fin_err_no_noise)

    l_delta_x.append(delta_x)
    l_delta_y.append(delta_y)
    l_delta_s.append(delta_s)

m_errors = np.array(l_errors).reshape(len(l_disp_g), len(l_add_noise_disp))
m_errors_no_noise = np.array(l_errors_no_noise).reshape(len(l_disp_g), len(l_add_noise_disp))

m_delta_x = np.array(l_delta_x).reshape(len(l_disp_g), len(l_add_noise_disp))
m_delta_y = np.array(l_delta_y).reshape(len(l_disp_g), len(l_add_noise_disp))
m_delta_s = np.array(l_delta_s).reshape(len(l_disp_g), len(l_add_noise_disp))


=== disp_g: 0.075	ampl_noise: 0	add_noise: 0.0 ===


C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

invalid value encountered in scalar divide



= best_fin_err: 7.470627724383522e-12 =
= best_fin_err_no_noise: 7.470627724383522e-12 =

=== disp_g: 0.075	ampl_noise: 0	add_noise: 0.0033333333333333335 ===
= best_fin_err: 0.010387282542651432 =
= best_fin_err_no_noise: 5.663920441199529e-05 =

=== disp_g: 0.075	ampl_noise: 0	add_noise: 0.006666666666666667 ===
= best_fin_err: 0.04286429743814785 =
= best_fin_err_no_noise: 0.00021950616665369096 =

=== disp_g: 0.075	ampl_noise: 0	add_noise: 0.01 ===
= best_fin_err: 0.09740118319623942 =
= best_fin_err_no_noise: 0.0008602906415997279 =

=== disp_g: 0.075	ampl_noise: 0	add_noise: 0.013333333333333334 ===
= best_fin_err: 0.16498107107860335 =
= best_fin_err_no_noise: 0.000679897121566098 =

=== disp_g: 0.075	ampl_noise: 0	add_noise: 0.016666666666666666 ===
= best_fin_err: 0.27186493478287665 =
= best_fin_err_no_noise: 0.0024505230668104457 =

=== disp_g: 0.075	ampl_noise: 0	add_noise: 0.02 ===
= best_fin_err: 0.40785697898277395 =
= best_fin_err_no_noise: 0.0027214202695426282 =

=== 

In [7]:
fig = px.imshow((m_errors),
                labels=dict(y="disp_g", x="add_noise_disp", color="Error"),
                y=l_disp_g,
                x=l_add_noise_disp)

fig.show()

fig = px.imshow((m_errors_no_noise),
                labels=dict(y="disp_g", x="add_noise_disp", color="Error"),
                y=l_disp_g,
                x=l_add_noise_disp)

fig.show()

In [8]:
trash_hold = 0.3
m_errors_1 = np.multiply(m_errors, (m_errors<trash_hold))
#m_errors_1 = np.log(m_errors_1/np.max(m_errors_1))


fig = px.imshow((m_errors_1),
                labels=dict(y="disp_g", x="add_noise_disp", color="Error"),
                y=l_disp_g,
                x=l_add_noise_disp)

fig.show()

In [9]:
fig = px.imshow((m_delta_x),
                labels=dict(y="disp_g", x="add_noise_disp", color="Error"),
                y=l_disp_g,
                x=l_add_noise_disp)

fig.show()

fig = px.imshow((m_delta_y),
                labels=dict(y="disp_g", x="add_noise_disp", color="Error"),
                y=l_disp_g,
                x=l_add_noise_disp)

fig.show()

fig = px.imshow((m_delta_s),
                labels=dict(y="disp_g", x="add_noise_disp", color="Error"),
                y=l_disp_g,
                x=l_add_noise_disp)

fig.show()

In [18]:
disp_g = 0.075#0.092
ampl_gaus_noise = 0
add_noise_disp = 0.04

init_sol = [0.3, 0.6, disp_g, 
            0.5, 0.8, disp_g,
            0.7, 0.7, disp_g,
            0.5, ampl_gaus_noise, 0.3]

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1],
          [0,1], [0,1], [0,1]]

sol, fin_err, fin_err_no_noise,x,y,d = mult_exper_3(disp_g, ampl_gaus_noise, bounds = bounds, add_noise_disp = add_noise_disp, num_launches=15)


print(f'fin_err: {fin_err}\fin_err_no_noise: {fin_err_no_noise}\tsol: {sol}')

init_spec, X = EvalSol(init_sol, add_noise_disp=add_noise_disp)
fin_spec, X = EvalSol(sol)
DrawApprox(init_spec, fin_spec, X, sol)
DrawApprox(init_spec, fin_spec, X, init_sol)


=== disp_g: 0.075	ampl_noise: 0	add_noise: 0.04 ===


C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

divide by zero encountered in scalar divide



= best_fin_err: 1.4709673090271003 =
= best_fin_err_no_noise: 0.0051527018797145555 =
fin_err: 1.4709673090271003in_err_no_noise: 0.0051527018797145555	sol: [0.3014665228679899, 0.6052374147620054, 0.07530928441158966, 0.49925081457659776, 0.7925904718740214, 0.07363790468333267, 0.6985383285414403, 0.7030300093350411, 0.07603476994754291]


In [19]:
disp_g = 0.084#0.092
ampl_gaus_noise = 0
add_noise_disp = 0.04

init_sol = [0.3, 0.6, disp_g, 
            0.5, 0.8, disp_g,
            0.7, 0.7, disp_g,
            0.5, ampl_gaus_noise, 0.3]

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1],
          [0,1], [0,1], [0,1]]

sol, fin_err, fin_err_no_noise,x,y,d = mult_exper_3(disp_g, ampl_gaus_noise, bounds = bounds, add_noise_disp = add_noise_disp, num_launches=15)


print(f'fin_err: {fin_err}\fin_err_no_noise: {fin_err_no_noise}\tsol: {sol}')

init_spec, X = EvalSol(init_sol, add_noise_disp=add_noise_disp)
fin_spec, X = EvalSol(sol)
DrawApprox(init_spec, fin_spec, X, sol)
DrawApprox(init_spec, fin_spec, X, init_sol)


=== disp_g: 0.084	ampl_noise: 0	add_noise: 0.04 ===


C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

divide by zero encountered in scalar divide



= best_fin_err: 1.554907484170571 =
= best_fin_err_no_noise: 0.003008406270120418 =
fin_err: 1.554907484170571in_err_no_noise: 0.003008406270120418	sol: [0.29828473114248166, 0.5975482044043, 0.0828630818590508, 0.4978853083263528, 0.7987442993332369, 0.08460070799674997, 0.6993073926600182, 0.6995645600164083, 0.08493191480156412]


In [20]:
disp_g = 0.093#0.092
ampl_gaus_noise = 0
add_noise_disp = 0.04

init_sol = [0.3, 0.6, disp_g, 
            0.5, 0.8, disp_g,
            0.7, 0.7, disp_g,
            0.5, ampl_gaus_noise, 0.3]

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1],
          [0,1], [0,1], [0,1]]

sol, fin_err, fin_err_no_noise,x,y,d = mult_exper_3(disp_g, ampl_gaus_noise, bounds = bounds, add_noise_disp = add_noise_disp, num_launches=15)


print(f'fin_err: {fin_err}\fin_err_no_noise: {fin_err_no_noise}\tsol: {sol}')

init_spec, X = EvalSol(init_sol, add_noise_disp=add_noise_disp)
fin_spec, X = EvalSol(sol)
DrawApprox(init_spec, fin_spec, X, sol)
DrawApprox(init_spec, fin_spec, X, init_sol)


=== disp_g: 0.093	ampl_noise: 0	add_noise: 0.04 ===


C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

invalid value encountered in scalar divide



= best_fin_err: 1.528624801256981 =
= best_fin_err_no_noise: 0.01657472153403848 =
fin_err: 1.528624801256981in_err_no_noise: 0.01657472153403848	sol: [0.30656143313232326, 0.6293829521846485, 0.09640412907701672, 0.5000075807085926, 0.7518663373383458, 0.08614006133404163, 0.6927539749366478, 0.7270739815313743, 0.09697736118511692]


In [21]:
disp_g = 0.114#0.092
ampl_gaus_noise = 0
add_noise_disp = 0.04

init_sol = [0.3, 0.6, disp_g, 
            0.5, 0.8, disp_g,
            0.7, 0.7, disp_g,
            0.5, ampl_gaus_noise, 0.3]

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1],
          [0,1], [0,1], [0,1]]

sol, fin_err, fin_err_no_noise,x,y,d = mult_exper_3(disp_g, ampl_gaus_noise, bounds = bounds, add_noise_disp = add_noise_disp, num_launches=15)


print(f'fin_err: {fin_err}\fin_err_no_noise: {fin_err_no_noise}\tsol: {sol}')

init_spec, X = EvalSol(init_sol, add_noise_disp=add_noise_disp)
fin_spec, X = EvalSol(sol)
DrawApprox(init_spec, fin_spec, X, sol)
DrawApprox(init_spec, fin_spec, X, init_sol)


=== disp_g: 0.114	ampl_noise: 0	add_noise: 0.04 ===


C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_25620\1965247583.py:3: RuntimeWarning:

divide by zero encountered in scalar divide



= best_fin_err: 1.4916261474775974 =
= best_fin_err_no_noise: 0.0343062473401371 =
fin_err: 1.4916261474775974in_err_no_noise: 0.0343062473401371	sol: [0.2926504285111291, 0.48533535527385535, 0.11254576152280239, 0.5239666209472363, 0.9696115524648495, 0.1431575603832888, 0.7318758081639316, 0.44040469281096656, 0.10184202830647976]
